# Palmer Archipelago (Antarctica) penguin data

![](https://allisonhorst.github.io/palmerpenguins/reference/figures/lter_penguins.png)

In [287]:
import numpy as np 
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt

## Leitura do Dataset

In [288]:
#Leitura do dataset
penguins = pd.read_csv("../input/palmer-archipelago-antarctica-penguin-data/penguins_size.csv")
#Observar algumas linhas do dataset
penguins.head()

## Exploração dos dados

In [289]:
penguins.columns

* **species**: espécies de pinguins
* **culmen_length_mm**: comprimento do *culmen* em milímetros
* **culmen_depth_mm**: comprimento do *culmen* em milímetros
* **flipper_length_mm**: comprimento da barbatana em milímetros
* **body_mass_g**: massa corporal em gramas
* **island**: nome da ilha no Arquipélago Palmer (Antártica)
* **sex**: sexo do pinguim

![](https://pbs.twimg.com/media/EaAXQn8U4AAoKUj?format=jpg&name=4096x4096)



In [290]:
penguins.info()

In [291]:
penguins.describe()

### Verificação da existência de valores em falta

In [292]:
#Verificar quais colunas têm valores em falta
penguins.isna().any()

In [293]:
#Verificar quantos valores em falta existem
penguins.isna().sum()

### Valores únicos

Verificação dos valores únicos para as *features* categóricas.

In [294]:
print(penguins['species'].unique())
print(penguins['island'].unique())
print(penguins['sex'].unique())

* Existem 3 espécies de pinguins: **Adelie**, **Chinstrap** e **Gentoo**
* Existem 2 ilhas no Arquipélago: **Torgersen**, **Biscoe** e **Dream**

### Groupby

In [295]:
penguins.groupby(by="species").mean()

In [296]:
penguins.groupby(by=['island','species']).mean()

Verificamos que:
* a espécie Gentoo apenas existe na ilha Biscoe;
* a espécie Chinstrap apenas existe na ilha Dream;
* a espécie Adelie está presente em todas as ilhas.

In [297]:
penguins['species'].value_counts()

Verificamos que existem muitos menos pinguins Chinstrap do que pinguins das outras espécies

### Distribuição

In [298]:
sns.histplot(penguins['culmen_length_mm'], kde=True)

In [299]:
print("Skewness: %f" % penguins['culmen_length_mm'].skew())
print("Kurtosis: %f" % penguins['culmen_length_mm'].kurt())

In [300]:
sns.histplot(penguins['culmen_depth_mm'], kde=True)

In [301]:
print("Skewness: %f" % penguins['culmen_depth_mm'].skew())
print("Kurtosis: %f" % penguins['culmen_depth_mm'].kurt())

In [302]:
sns.histplot(penguins['flipper_length_mm'], kde=True)

In [303]:
print("Skewness: %f" % penguins['flipper_length_mm'].skew())
print("Kurtosis: %f" % penguins['flipper_length_mm'].kurt())

In [304]:
sns.histplot(penguins['body_mass_g'], kde=True)

In [305]:
print("Skewness: %f" % penguins['body_mass_g'].skew())
print("Kurtosis: %f" % penguins['body_mass_g'].kurt())

### Outliers

In [306]:
sns.boxplot(x=penguins['culmen_length_mm'])

In [307]:
sns.boxplot(x=penguins['culmen_length_mm'], y=penguins["species"])

Apesar de na *boxplot* da *culmen_length_mm* global não existirem outliers, verificamos que, fazendo um *boxplot* desta *feature* para cada espécie, **existe um outlier nos pinguins da espécie Gentoo**.

In [308]:
sns.boxplot(x=penguins['culmen_depth_mm'])

In [309]:
sns.boxplot(x=penguins['culmen_depth_mm'], y=penguins['species'])

Apesar de na *boxplot* da *culmen_depth_mm* global não existirem outliers, verificamos que, fazendo um *boxplot* desta *feature* para cada espécie, **existe um outlier nos pinguins da espécie Adelie**.

In [310]:
sns.boxplot(x=penguins['flipper_length_mm'])

In [311]:
sns.boxplot(x=penguins['flipper_length_mm'], y=penguins['species'])

Apesar de na *boxplot* da *flipper_length_mm* global não existirem outliers, verificamos que, fazendo um *boxplot* desta *feature* para cada espécie, **existem dois outliers nos pinguins da espécie Adelie**.

In [312]:
sns.boxplot(x=penguins['body_mass_g'])

In [313]:
sns.boxplot(x=penguins['body_mass_g'], y=penguins['species'])

Apesar de na *boxplot* da *body_mass_g* global não existirem outliers, verificamos que, fazendo um *boxplot* desta *feature* para cada espécie, **existem dois outliers nos pinguins da espécie Chinstrap**.

### Scatter plots

In [314]:
sns.pairplot(penguins)

### Matriz de correlação

In [315]:
corr_matrix = penguins.corr()
f, ax = plt.subplots(figsize=(8,6))
sns.heatmap(corr_matrix,vmin=-1,vmax=1,square=True,annot=True)

## Preparação dos dados

### Transformação dos dados categóricos

In [318]:
penguins['island'].replace({'Biscoe':0, 'Dream': 1, 'Torgersen':2}, inplace=True)
penguins['species'].replace({'Adelie':0, 'Chinstrap': 1, 'Gentoo':2}, inplace=True)
penguins['sex'].replace({'MALE':0, 'FEMALE': 1}, inplace=True)
penguins.rename(columns={'sex':'female'}, inplace=True)

penguins.head()